In [1]:
from google.colab import drive
drive.mount('/gdrive')

drivePath = '/gdrive/MyDrive/MSCE/FrameGate'

Mounted at /gdrive


In [3]:

!unzip /gdrive/MyDrive/MSCE/FrameGate/frames.zip

Streaming output truncated to the last 5000 lines.
  inflating: frames/Sitting/Sitting (53)/frame_019.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_033.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_027.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_219.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_231.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_225.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_184.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_190.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_147.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_153.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_152.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_146.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_191.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_185.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_224.jpg  
  inflating: frames/Sitting/Sitting (53)/frame_230.jpg  
  inflating: frames/Sitting/Sitting (

In [9]:
import shutil
shutil.copy(f'{drivePath}/cleaned_impact.csv.zip', ".")

!unzip impact.csv.zip
!unzip cleaned_impact.csv.zip
!mv cleaned_impact.csv impact.csv

unzip:  cannot find or open impact.csv.zip, impact.csv.zip.zip or impact.csv.zip.ZIP.
Archive:  cleaned_impact.csv.zip
  inflating: cleaned_impact.csv      


In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("sharjeelmazhar/human-activity-recognition-video-dataset")

# print("Path to dataset files:", path)


100%|██████████| 14.8G/14.8G [02:52<00:00, 92.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sharjeelmazhar/human-activity-recognition-video-dataset/versions/1


In [ ]:
# !ls '/root/.cache/kagglehub/datasets/sharjeelmazhar/human-activity-recognition-video-dataset/versions/1/Human Activity Recognition - Video Dataset'

 Clapping	   Sitting	     Walking			  'Walking While Using Phone'
'Meet and Split'  'Standing Still'  'Walking While Reading Book'


In [ ]:
# # move the dataset to the correct location
# import shutil
# shutil.move(path, "..")

'../1'

In [ ]:

# # rename the directory in data to remove the spaces
# shutil.move("../1/Human Activity Recognition - Video Dataset", "../HumanActivityRecognition-VideoDataset")

'../HumanActivityRecognition-VideoDataset'

In [ ]:
# shutil.move("../HumanActivityRecognition-VideoDataset/Meet and Split", "../HumanActivityRecognition-VideoDataset/MeetAndSplit")
# shutil.move("../HumanActivityRecognition-VideoDataset/Standing Still", "../HumanActivityRecognition-VideoDataset/StandingStill")
# shutil.move("../HumanActivityRecognition-VideoDataset/Walking While Reading Book", "../HumanActivityRecognition-VideoDataset/WalkingWhileReadingBook")
# shutil.move("../HumanActivityRecognition-VideoDataset/Walking While Using Phone", "../HumanActivityRecognition-VideoDataset/WalkingWhileUsingPhone")


'../HumanActivityRecognition-VideoDataset/WalkingWhileUsingPhone'

In [5]:
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
import tqdm
import matplotlib.pyplot as plt

In [10]:

# Read the CSV file
csv_file_path = 'impact.csv'
data = pd.read_csv(csv_file_path)

In [11]:
import os
# ../HumanActivityRecognition-VideoDataset/StandingStill/Standing Still (19).mp4,0,1,0.27592835
# Frame n: 0, Frame m: 1, Perceived Change: 0.27592835
# Frame n would be found at ../frames/StandingStill/Standing Still (19)/frame_000.jpg
# Frame m would be found at ../frames/StandingStill/Standing Still (19)/frame_001.jpg
# Function to load an image from a file
def load_image(image_path, image_size=(224, 224)):
    image = cv2.imread(image_path)
    # image = cv2.resize(image, image_size)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # image = image / 255.0
    return image

# Refactored load_data function
def load_data(csv_data, frames_directory):
    frames_n_list = []
    frames_m_list = []
    perceived_changes = []

    for index, row in tqdm.tqdm(csv_data.iterrows(), total=csv_data.shape[0], desc="Loading data"):
        video_path = row['Video']
        frame_n_index = int(row['Frame n'])
        frame_m_index = int(row['Frame m'])
        perceived_change = row['Perceived Change']

        # Construct the paths to the frame images
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        video_dir = os.path.dirname(video_path).split('/')[-1]
        frame_n_path = os.path.join(frames_directory, video_dir, video_name, f"frame_{frame_n_index:03d}.jpg")
        frame_m_path = os.path.join(frames_directory, video_dir, video_name, f"frame_{frame_m_index:03d}.jpg")

        # Load the frames from the image files
        frame_n = load_image(frame_n_path)
        frame_m = load_image(frame_m_path)

        frames_n_list.append(frame_n)
        frames_m_list.append(frame_m)
        perceived_changes.append(perceived_change)

    return frames_n_list, frames_m_list, perceived_changes

In [ ]:
frames_n, frames_m, perceived_changes = load_data(data, frames_directory='frames')

# Print the first n and m frame and the perceived change
print(perceived_changes[0])
plt.imshow(frames_n[0])
plt.show()
plt.imshow(frames_m[0])
plt.show()


Loading data:  11%|█         | 36161/341557 [03:16<27:59:07,  3.03it/s]

In [ ]:
# ! ls ../HumanActivityRecognition-VideoDataset/

Clapping      Sitting	     Walking		      WalkingWhileUsingPhone
MeetAndSplit  StandingStill  WalkingWhileReadingBook


In [ ]:
# normalize the images
frames_n = np.array(frames_n) / 255.0
frames_m = np.array(frames_m) / 255.0
# Calculate the minimum and maximum values
min_value = np.min(perceived_changes)
max_value = np.max(perceived_changes)
# Apply Min-Max normalization
perceived_changes = (perceived_changes - min_value) / (max_value - min_value)

In [ ]:
# Shuffle the data
indices = np.arange(len(perceived_changes))
np.random.shuffle(indices)

frames_n = tf.gather(frames_n, indices)
frames_m = tf.gather(frames_m, indices)
perceived_changes = tf.gather(perceived_changes, indices)

In [ ]:
# Split the data into training, validation, and test sets
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

# Calculate the split indices
num_samples = len(perceived_changes)
train_end = int(train_ratio * num_samples)
val_end = train_end + int(val_ratio * num_samples)

# Split the data
frames_n_train = frames_n[:train_end]
frames_m_train = frames_m[:train_end]
perceived_changes_train = perceived_changes[:train_end]

frames_n_val = frames_n[train_end:val_end]
frames_m_val = frames_m[train_end:val_end]
perceived_changes_val = perceived_changes[train_end:val_end]

frames_n_test = frames_n[val_end:]
frames_m_test = frames_m[val_end:]
perceived_changes_test = perceived_changes[val_end:]




In [ ]:
frames_m_test.shape

TensorShape([6, 224, 224, 3])

In [ ]:
# Create a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(((frames_n_train, frames_m_train), perceived_changes_train))
val_dataset = tf.data.Dataset.from_tensor_slices(((frames_n_val, frames_m_val), perceived_changes_val))
test_dataset = tf.data.Dataset.from_tensor_slices(((frames_n_test, frames_m_test), perceived_changes_test))

# Shuffle and batch the dataset
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=len(perceived_changes)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Define the neural network model
input_n = tf.keras.layers.Input(shape=(224, 224, 3))
input_m = tf.keras.layers.Input(shape=(224, 224, 3))

# Shared convolutional base
base_model = tf.keras.applications.MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
x_n = base_model(input_n)
x_m = base_model(input_m)

# Global average pooling
x_n = tf.keras.layers.GlobalAveragePooling2D()(x_n)
x_m = tf.keras.layers.GlobalAveragePooling2D()(x_m)

# Concatenate the outputs
x = tf.keras.layers.Concatenate()([x_n, x_m])

# Fully connected layer
x = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=[input_n, input_m], outputs=output)



4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model.trainable_variables


[<KerasVariable shape=(1152, 128), dtype=float32, path=dense/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=dense/bias>,
 <KerasVariable shape=(128, 1), dtype=float32, path=dense_1/kernel>,
 <KerasVariable shape=(1,), dtype=float32, path=dense_1/bias>]

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ MobileNetV3Small          │ (None, 7, 7, 576)      │        939,120 │ input_layer[0][0],     │
│ (Functional)              │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 576)            │              0 │ MobileNetV3Small[0][0] │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 576)            │              0 │ MobileNetV3Small[1][0] │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 1152)           │              0 │ global_average_poolin… │
│                           │                        │                │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │        147,584 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │            129 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,086,833 (4.15 MB)

 Trainable params: 147,713 (577.00 KB)

 Non-trainable params: 939,120 (3.58 MB)

In [ ]:
# Train the new layers for a few epochs
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)

# Save the trained model
model.save(f'{drivePath}/trained_model.h5')

In [ ]:
# Unfreeze the last 50% of the layers of the base model
total_layers = len(base_model.layers)
for layer in base_model.layers[int(total_layers * 0.5):]:
    layer.trainable = True

In [ ]:
def lr_schedule(epoch, lr):
    decay_rate = 0.1
    decay_step = 5
    if epoch % decay_step == 0 and epoch:
        return lr * decay_rate
    return lr

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Define the model checkpoint callback
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f'{drivePath}/model_checkpoint.keras',  # File path to save the model
    monitor='val_loss',     # Monitor the validation loss
    save_best_only=True,    # Save only the best model
    save_weights_only=False, # Save the entire model (architecture + weights)
    mode='min',             # Save the model with the minimum validation loss
    verbose=1               # Print a message when saving the model
)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 100
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), early_stopping, checkpoint]
)

# Save the trained model again after unfreezing and further training
model.save(f'{drivePath}/fine_tuned_model.h5')


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - loss: 0.3117 - mae: 0.4474 - val_loss: 0.7570 - val_mae: 0.8414
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - loss: 0.5848 - mae: 0.6840 - val_loss: 0.3708 - val_mae: 0.5674
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - loss: 0.2850 - mae: 0.4333 - val_loss: 0.0555 - val_mae: 0.2163
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step - loss: 0.1216 - mae: 0.3160 - val_loss: 0.1178 - val_mae: 0.2633
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step - loss: 0.2918 - mae: 0.4310 - val_loss: 0.1361 - val_mae: 0.2960
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 574ms/step - loss: 0.3202 - mae: 0.4546 - val_loss: 0.0595 - val_mae: 0.1862
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step - loss: 0.1841 - mae: 0.3481 - val_loss: 0.0795 - val_mae: 0.2628
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - loss: 0.1165 - mae: 0.3152 - val_loss: 0.2196 - val_mae: 0.4133
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step - loss: 0.1821 - mae: 0

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 0.1485 - mae: 0.3309
Test loss: 0.14848117530345917
Test MAE: 0.33092865347862244
